In [1]:
!kaggle datasets download -d abhi8923shriv/sentiment-analysis-dataset

Dataset URL: https://www.kaggle.com/datasets/abhi8923shriv/sentiment-analysis-dataset
License(s): CC0-1.0
sentiment-analysis-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import zipfile
zip_ref=zipfile.ZipFile('/content/sentiment-analysis-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [3]:
import pandas as pd

df = pd.read_csv('/content/train.csv', encoding='latin-1',dtype={'text': str})
df['text'].fillna(" ",inplace=True)
from sklearn.model_selection import train_test_split
train,test=train_test_split(df,test_size=0.2,random_state=42)
y_train=train['sentiment']
y_test=test['sentiment']
x_train=train['text']
x_test=test['text']


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27481 entries, 0 to 27480
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   textID            27481 non-null  object 
 1   text              27481 non-null  object 
 2   selected_text     27480 non-null  object 
 3   sentiment         27481 non-null  object 
 4   Time of Tweet     27481 non-null  object 
 5   Age of User       27481 non-null  object 
 6   Country           27481 non-null  object 
 7   Population -2020  27481 non-null  int64  
 8   Land Area (Km²)   27481 non-null  float64
 9   Density (P/Km²)   27481 non-null  int64  
dtypes: float64(1), int64(2), object(7)
memory usage: 2.1+ MB


In [5]:
df['sentiment'].value_counts()

sentiment
neutral     11118
positive     8582
negative     7781
Name: count, dtype: int64

In [6]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
Tokenizer= Tokenizer(oov_token='<Happy>')

In [7]:
x_train.fillna(" ",inplace=True)
x_test.fillna(" ",inplace=True)

train_data=Tokenizer.fit_on_texts(x_train)
t_data=Tokenizer.fit_on_texts(x_test)

In [8]:
Tokenizer.document_count


27481

In [9]:
sequence=Tokenizer.texts_to_sequences(x_train)
test_seq=Tokenizer.texts_to_sequences(x_test)

In [10]:
len(Tokenizer.word_counts)


26598

In [11]:
sq=pad_sequences(sequence,padding='post',maxlen=50)
y_train=train['sentiment']
test_sq=pad_sequences(test_seq,padding='post',maxlen=50)
y_test=test['sentiment']

In [12]:

sq.shape

(21984, 50)

In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import SimpleRNN , Dense ,Embedding,Flatten,Dropout

In [14]:
model=Sequential()

model.add(SimpleRNN(32,input_shape=[50,1],return_sequences=False))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3,activation='sigmoid'))

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 32)                1088      
                                                                 
 dense (Dense)               (None, 128)               4224      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 3)                 387       
                                                                 
Total params: 5699 (22.26 KB)
Trainable params: 5699 (22.26 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [17]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)
y_tr = to_categorical(y_train_encoded)
y_te = to_categorical(y_test_encoded)

model.fit(sq,y_tr,epochs=10,validation_data=(test_sq,y_te))

Epoch 1/10
687/687 [==============================] - 18s 17ms/step - loss: 1.0903 - accuracy: 0.3991 - val_loss: 1.0866 - val_accuracy: 0.4057
Epoch 2/10
687/687 [==============================] - 11s 16ms/step - loss: 1.0872 - accuracy: 0.4042 - val_loss: 1.0863 - val_accuracy: 0.4057
Epoch 3/10
687/687 [==============================] - 10s 14ms/step - loss: 1.0870 - accuracy: 0.4037 - val_loss: 1.0863 - val_accuracy: 0.4057
Epoch 4/10
687/687 [==============================] - 12s 17ms/step - loss: 1.0873 - accuracy: 0.4037 - val_loss: 1.0869 - val_accuracy: 0.4057
Epoch 5/10
687/687 [==============================] - 11s 16ms/step - loss: 1.0871 - accuracy: 0.4045 - val_loss: 1.0867 - val_accuracy: 0.4057
Epoch 6/10
687/687 [==============================] - 11s 16ms/step - loss: 1.0873 - accuracy: 0.4038 - val_loss: 1.0893 - val_accuracy: 0.4057
Epoch 7/10
687/687 [==============================] - 11s 17ms/step - loss: 1.0873 - accuracy: 0.4037 - val_loss: 1.0865 - val_accuracy:

In [18]:
from sklearn.metrics import confusion_matrix
import pandas as pd
check=pd.read_csv('/content/test.csv',encoding='latin-1',dtype={'text':str})
check=check[~ check['text'].isnull()]
testing=check['text']
testing=Tokenizer.texts_to_sequences(testing)
sqe=pad_sequences(testing,padding='post',maxlen=50)
y_testing=check['sentiment']
y_test_pred=model.predict(sqe)
t=pd.DataFrame(y_test_pred,columns=['neutral',"positive",'negaitve'])
classes=[]
for i in range(len(t)):
   for j in range (3):
    if t.iloc[i,j]==t.iloc[i].max():
      t.iloc[i,j]=1
      classes.append(t.columns[j])
    else:
      t.iloc[i,j]=0
check['sentiment']
matrix=confusion_matrix(check['sentiment'],classes)
matrix

111/111 [==============================] - 1s 5ms/step


array([[   0,    0, 1001],
       [   0,    0, 1430],
       [   0,    0, 1103]])

In [19]:

#Embedding

In [32]:
model1=Sequential()
model1.add(Embedding(26598,output_dim=2,input_length=50))
model1.add(SimpleRNN(32,return_sequences=False))
model1.add(Dense(128,activation='relu'))
model1.add(Dropout(0.2))
model1.add(Dense(3,activation='sigmoid'))

In [28]:
model1.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 2)             53196     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 32)                1120      
                                                                 
 dense_4 (Dense)             (None, 128)               4224      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_5 (Dense)             (None, 3)                 387       
                                                                 
Total params: 58927 (230.18 KB)
Trainable params: 58927 (230.18 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [29]:
model1.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [30]:

model1.fit(sq,y_tr,epochs=10,validation_data=(test_sq,y_te))

Epoch 1/10
687/687 [==============================] - 47s 66ms/step - loss: 1.0548 - accuracy: 0.4405 - val_loss: 0.9812 - val_accuracy: 0.5025
Epoch 2/10
687/687 [==============================] - 30s 44ms/step - loss: 0.9311 - accuracy: 0.5373 - val_loss: 0.8924 - val_accuracy: 0.5769
Epoch 3/10
687/687 [==============================] - 28s 41ms/step - loss: 0.8126 - accuracy: 0.6397 - val_loss: 0.9345 - val_accuracy: 0.5638
Epoch 4/10
687/687 [==============================] - 28s 41ms/step - loss: 0.6960 - accuracy: 0.7153 - val_loss: 0.9002 - val_accuracy: 0.6125
Epoch 5/10
687/687 [==============================] - 28s 40ms/step - loss: 0.6115 - accuracy: 0.7665 - val_loss: 0.8848 - val_accuracy: 0.6371
Epoch 6/10
687/687 [==============================] - 28s 41ms/step - loss: 0.5415 - accuracy: 0.8034 - val_loss: 0.9223 - val_accuracy: 0.6333
Epoch 7/10
687/687 [==============================] - 27s 40ms/step - loss: 0.4875 - accuracy: 0.8281 - val_loss: 0.9203 - val_accuracy:

In [24]:
y_test_pred=model1.predict(sqe)
test=pd.DataFrame(y_test_pred,columns=['neutral',"positive",'negaitve'])
classes=[]
for i in range(len(test)):
   for j in range (3):
    if test.iloc[i,j]==test.iloc[i].max():
      test.iloc[i,j]=1
      classes.append(test.columns[j])
    else:
      test.iloc[i,j]=0

matrix=confusion_matrix(check['sentiment'],classes)
matrix

111/111 [==============================] - 1s 5ms/step


array([[  0,   0,   0,   0],
       [ 49,   0, 547, 405],
       [205,   0, 286, 939],
       [714,   0,  91, 298]])